In [1]:
%%file Q3.py
import numpy as np
from numpy import sqrt, sin, cos, pi, tan
from collections import deque
from mpi4py import MPI
from  time import time
import math
import sys 
"""
This code uses the 3-D parallelized adaptive algorithm to calculate the integral of a funciton f.
tol is the tolerance.
f is the function of integrand.
result is the approximate result of the integration.
errorEstimate is the error estimated.
finestLevel is the finest grid in the interval.

Some part of the code could be referred to the following link. 
https://github.com/stefantaylor/Parallel-Programming-Languages-and-Systems/blob/master/cw2/aquadPartA.c

Use the trilinear transformation method to map an arbitrary haxahedron into a referenced unitary cube. 

Then, use the parallelized 3-D adaptive Simpson algorithm to get the integral.

A stack is used to store the sub-interval of the domain and sub-tolerance. It is initilized with [X1, X2, Y1, Y2, Z1, Z2, tol].

A farmer cpu is used to monitor the other worker cpu, which are used to calculate the integral or divide the interval.

The main idea of the algorithm is as follows:
1.  A worker cpu calculate the integral I1 and I2 by using Simpson rule and composite Simpson rule
2.  If the difference between them is smaller than the tolrance, it will return I2 to the farmer cpu.
    If not, it will divide the domain by 8, and send the interval to farmer cpu. This worker cpu will be idle.
3.  The farmer cpu pop a interval from the stack, and sends it to a worker cpu which is idle.
4.  After receiving the sub-interval from the worker cpus, the farmer cpu will add them to the stack.  

"""

comm = MPI.COMM_WORLD
rank = comm.rank
numprocs = comm.size

global X1, X2, Y1, Y2, Z1, Z2, tol, maxLevels, AtoH

# the boundary on the unitary cube 
# Do not change X1, X2, Y1, Y2, Z1, Z2 !!!
X1 = 0.0; X2 = 1.0
Y1 = 0.0; Y2 = 1.0
Z1 = 0.0; Z2 = 1.0    

# set the tolerance and maxLevels
tol = 0.0001
maxLevels = 200000

# the eight vertices of the arbitrary domain.
# The order of coordinates of the eight vertices should be strictly given according to the reference.
v000 = [0, 0, 0]
v100 = [2, 0, 0]
v110 = [1.5, 1, 0]
v010 = [0.5, 1, 0]
v001 = [0, 0, 2]
v101 = [2, 0, 2]
v111 = [1.5, 1, 2]
v011 = [0.5, 1, 2]

# f(x) is the function to be integrated. Several candidate functions are shown here
def f(x, y, z):
    return x*y*(z**0.8) +sin(x)+sin(y)
#     return sin(z)
#     return z

def farmer(numprocs, maxLevels):
    myStack = deque()
    points = [X1, X2, Y1, Y2, Z1, Z2, tol]
    myStack.append(points)
    worker = []
    newdata =[]
    status = MPI.Status()
    numberLevels = 0
    finestLevel = X2-X1
    for i in range(numprocs):
        worker.append(0)
    total = 0
    errorEstimate = 0

    while numberLevels <= maxLevels:  # while numberLevels <= maxLevels, do the loop
        numberLevels = 1+numberLevels
        for i in range(1, numprocs):
            
# if the worker cpu is idle and the stack is not empty, pop it from the stack and send it to the worker cpu
            if worker[i] == 0 and len(myStack) > 0:
                data = myStack.pop()
                worker[i] = 1;
                comm.send(data, dest = i, tag = 1 )
        # receive data from the worker cpus
        newdata = comm.recv(source = MPI.ANY_SOURCE, tag = MPI.ANY_TAG, status=status)
        source = status.Get_source()
        tag = status.Get_tag()
        
        # tag =1 means the interval is too big and the error of integration is larger than tolerance. 
        # we need to divide the interval by eight
        if tag == 1:
            numberLevels = numberLevels-1
            myStack.append([newdata[0], newdata[1], newdata[3], newdata[4], newdata[6], newdata[7], newdata[9]])
            myStack.append([newdata[1], newdata[2], newdata[3], newdata[4], newdata[6], newdata[7], newdata[9]])
            myStack.append([newdata[0], newdata[1], newdata[4], newdata[5], newdata[6], newdata[7], newdata[9]])
            myStack.append([newdata[1], newdata[2], newdata[4], newdata[5], newdata[6], newdata[7], newdata[9]])
            
            myStack.append([newdata[0], newdata[1], newdata[3], newdata[4], newdata[7], newdata[8], newdata[9]])
            myStack.append([newdata[1], newdata[2], newdata[3], newdata[4], newdata[7], newdata[8], newdata[9]])
            myStack.append([newdata[0], newdata[1], newdata[4], newdata[5], newdata[7], newdata[8], newdata[9]])
            myStack.append([newdata[1], newdata[2], newdata[4], newdata[5], newdata[7], newdata[8], newdata[9]])
            
            if (newdata[1]-newdata[0]) < finestLevel:
                finestLevel = newdata[1]-newdata[0]
                
        # error of integration is smaller than tolerance. We can add the integral to the total
        else:
            total = total + newdata[0]
            errorEstimate = errorEstimate+ newdata[1]
        worker[source] = 0

        flag = 0
        for i in range(1, numprocs):
            if worker[i] ==1:
                flag =1
                break
                
        # flag=0, and stack is empty. The calculation will end
        if flag == 0  and len(myStack) == 0:
            break
            
    # set all the tag=9, which means to end the calculation 
    for i in range(1, numprocs):
        comm.send(data, dest = i, tag = 9 )
    return total, numberLevels, errorEstimate, finestLevel


def worker(rank):
    status = MPI.Status()
    newdata = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    data = [0, 0, 0, 0, 0, 0, 0]
    while 1:
        data = comm.recv(source=0, tag= MPI.ANY_TAG, status=status) 
        tag = status.Get_tag()
        if tag == 9 :
            break
        else:
            x1 = data[0]
            x2 = data[1]
            y1 = data[2]
            y2 = data[3]
            z1 = data[4]
            z2 = data[5]
            tol = data[6]
            hx = data[1]-data[0]
            hy = data[3]-data[2]
            hz = data[5]-data[4]
            xmid = (x1+x2)/2
            ymid = (y1+y2)/2
            zmid = (z1+z2)/2
            # calculate coarse grid by  Simpson rule            
            wCoarse = np.array([1, 4, 1])
            nCoarse= wCoarse.size
            xGridCoarse = np.linspace(x1, x2, num=nCoarse)  
            yGridCoarse = np.linspace(y1, y2, num=nCoarse)  
            zGridCoarse = np.linspace(z1, z2, num=nCoarse)  
            I1 = 0
            for i in range(nCoarse):
                for j in range(nCoarse):
                    for k in range(nCoarse):
                        x, y, z = rMap(AtoH, xGridCoarse[k], yGridCoarse[j], zGridCoarse[i])
                        I1 = I1 + wCoarse[i]*wCoarse[j]*wCoarse[k]*f(x,y,z)*Jacobian(AtoH, xGridCoarse[k], yGridCoarse[j], zGridCoarse[i]) 
            I1 = I1*hx*hy*hz/216
            
            # calculate fine grid by composite Simpson rule 
            wFine = np.array([1, 4, 2, 4, 1])
            nFine = wFine.size
            xGridFine = np.linspace(x1, x2, num=nFine)  
            yGridFine = np.linspace(y1, y2, num=nFine)  
            zGridFine = np.linspace(z1, z2, num=nFine)  
            I2 = 0
            for i in range(nFine):
                for j in range(nFine):
                    for k in range(nFine):
                        x, y, z = rMap(AtoH, xGridFine[k], yGridFine[j], zGridFine[i])
                        I2 = I2 + wFine[i]*wFine[j]*wFine[k]*f(x,y,z)*Jacobian(AtoH, xGridFine[k], yGridFine[j], zGridFine[i]) 
            I2 = I2*hx*hy*hz/216/8
            if abs(I2-I1 ) >= 15*tol :
                newdata[0] = x1
                newdata[1] = xmid
                newdata[2] = x2
                newdata[3] = y1
                newdata[4] = ymid
                newdata[5] = y2
                newdata[6] = z1
                newdata[7] = zmid
                newdata[8] = z2
                newdata[9] = tol/8
                comm.send(newdata, dest = 0, tag = 1 )
            else:
                newdata[0] = I2
                newdata[1] = (I2-I1)/15
                comm.send(newdata, dest = 0, tag = 2 )
    
# output the A, B, C, D, E, F, G, H vector
def ABCDEFGH(v000, v100, v110, v010, v001, v101, v111, v011):
    Q =  np.array([v000, v100, v110, v010, v001, v101, v111, v011])
    A = Q[0]
    B = Q[1] - A
    C = Q[3] - A
    D = Q[4] - A
    E = Q[2] - A - B - C
    F = Q[5] - A - B - D
    G = Q[7] - A - C - D
    H = Q[6] - A - B - C - D - E - F - G
    AtoH = np.array([A, B, C, D, E, F, G, H])
    return AtoH

# mapping the pqr coordinates to xyz coordinate
def rMap(AtoH, p, q, r):
    xyz = AtoH[0] + AtoH[1]*p+AtoH[2]*q+AtoH[3]*r+AtoH[4]*p*q+AtoH[5]*p*r+AtoH[6]*q*r+AtoH[7]*p*q*r
    return xyz[0],xyz[1],xyz[2]

# calculate the determinant of trilinear Jacobian matrix
def Jacobian(AtoH, p, q, r):
    B = AtoH[1]
    C = AtoH[2]
    D = AtoH[3]
    E = AtoH[4]
    F = AtoH[5]
    G = AtoH[6]
    H = AtoH[7]
    detJ = (B[0]+E[0]*q + F[0]*r + H[0]*q*r)*(C[1] + E[1]*p +G[1]*r + H[1]*p*r)*(D[2] + F[2]*p + G[2]*q + H[2]*p*q) -\
            (B[0] + E[0]*q + F[0]*r + H[0]*q*r)*(D[1] + F[1]*p + G[1]*q + H[1]*p*q)*(C[2] + E[2]*p + G[2]*r + H[2]*p*r) -\
            (C[0] + E[0]*p + G[0]*r + H[0]*p*r)*(B[1] + E[1]*q + F[1]*r + H[1]*q*r)*(D[2] + F[2]*p + G[2]*q + H[2]*p*q) +\
            (C[0] + E[0]*p + G[0]*r + H[0]*p*r)*(D[1] + F[1]*p + G[1]*q + H[1]*p*q)*(B[2] + E[2]*q + F[2]*r + H[2]*q*r) +\
            (D[0] + F[0]*p + G[0]*q + H[0]*p*q)*(B[1] + E[1]*q + F[1]*r + H[1]*q*r)*(C[2] + E[2]*p + G[2]*r + H[2]*p*r) -\
            (D[0] + F[0]*p + G[0]*q + H[0]*p*q)*(C[1] + E[1]*p + G[1]*r + H[1]*p*r)*(B[2] + E[2]*q + F[2]*r + H[2]*q*r) 
    return detJ  

# get A, B, C, D, E, F, G, H vector
AtoH = ABCDEFGH(v000, v100, v110, v010, v001, v101, v111, v011)
    
if numprocs < 2:
    sys.exit("ERROR: Must have at least 2 processes to run") 

if rank == 0:
    result, numberLevels, errorEstimate, finestLevel  = farmer(numprocs, maxLevels)
else:
    worker(rank)

if rank == 0:
    print("result:         " + str(result))
    print("numberLevels:   " + str(numberLevels))
    print("errorEstimate:  " + str(errorEstimate))
    print("finestLevel:    " + '1/'+  str((X2-X1)/finestLevel) + '*(X2-X1)')


Overwriting Q3.py


In [2]:
!mpiexec -np 6 python Q3.py

result:         4.796617241430789
numberLevels:   890
errorEstimate:  -6.859670318273368e-06
finestLevel:    1/32.0*(X2-X1)
